In [1]:
import os
import math
import gc

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
application_train = pd.read_csv('../data/application_train.csv')
application_test = pd.read_csv('../data/application_test.csv')
bureau = pd.read_csv('../data/bureau.csv')
bureau_balance = pd.read_csv('../data/bureau_balance.csv')
credit_card_balance = pd.read_csv('../data/credit_card_balance.csv')
installments_payments = pd.read_csv('../data/installments_payments.csv')
POS_CASH_balance = pd.read_csv('../data/POS_CASH_balance.csv')
previous_application = pd.read_csv('../data/previous_application.csv')

In [ ]:
for t, row in zip(application_train.head().iterrows(), application_train.columns):
    print(t)
    print(row)
#     print(row['TARGET'])


In [ ]:
file_names = os.listdir('../data')[1:]
file_names.remove('application_train.csv')
file_names.insert(0, 'application_train.csv')

print(file_names)

In [ ]:
df_list = [application_train, application_test, bureau, bureau_balance, credit_card_balance, installments_payments, POS_CASH_balance ,previous_application]

In [ ]:
for file_name_and_file in zip(file_names, df_list):
    print('{} shape: {}'.format(file_name_and_file[0], np.shape(file_name_and_file[1])))

# 1. application_train analysis

In [ ]:
application_train.head()

In [ ]:
def missing_value_counter(DataFrame):
    print('The following features are missing values: \n')
    for column in DataFrame.columns:
        if DataFrame[column].isna().value_counts()[0] < len(DataFrame):
            no_missing = len(DataFrame) - DataFrame[column].isna().value_counts()[0]
            print('{} missing value counts: {}, {:.5f}% of column missing'.format(column, no_missing, no_missing/len(DataFrame[column])))

In [ ]:
missing_value_counter(application_train)

Lets first look at the distribution of the target

In [ ]:
plt.subplots(figsize = (15, 8))
plt.hist(application_train['TARGET'])
plt.ylabel('Frequency')
plt.xlabel('Target')
plt.title('Distribution of Target')

There is a clear class imbalance in the target so the model will be trained on skewed data which contains mostly records of people that could not pay back their loans. This means accuracy likely won't be a great metric also so AUROC, as will be used to later evaluate the model, is a better metric instead.

Plotting correlation between some of the numerical columns in the training dataset:

In [ ]:
heatmap_list = ['TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
                'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
                'OWN_CAR_AGE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

In [ ]:
def correlation_map(df, columns, figsize=(15,10)):
    correlation = (df.loc[:, columns]).corr()

    fig, ax = plt.subplots(figsize = figsize)
    sns.heatmap(correlation, annot = True, ax = ax)

In [ ]:
correlation_map(application_train, heatmap_list)

The three external sources features show some correlation with a number of features which means those features may be used to calculate these external source ratings. There is also some correlation between CNT_CHILDREN and DAYS_BIRTH. Interesting to note that REGION_RATING_CLIENT and REGION_RATING_CLIENT_W_CITY show a relatively high negative correlation with REGION_POPULATION_RELATIVE - meaning that the rating of the client's region is generally lower (I'm assuming this means better) the more people live in that region.

Looking at the correlation of each of the features with the target:

In [ ]:
target_corr = application_train.corr()['TARGET'].sort_values()

In [ ]:
print(target_corr.head(20))
print()
print(target_corr.tail(20))

The three external ratings EXT_SOURCE_1, 2, and 3 are somewhat negatively correlated with the target and the most positively correlated feature is REGION_RATING_CLIENT_W_CITY. However, none of the features have any significant correlation with the target.

From above, we know that the EXT_SOURCE features all contain a varying range of missing values with EXT_SOURCE_2 having, by far, the lowest number of NaN values

Looking at the correlation between the external sources, the target, and some other features:

In [ ]:
correlation_map(application_train, ['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'], (6,6))

EXT_SOURCE_1 and DAYS_BIRTH show a correlation meaning that age is likely a factor in the creation of the EXT_SOURCE_1 rating for applicants. These features will all likely be important for our model in prediction. Looking at a pairplot of these features:

In [ ]:
# Copy the data for plotting
plot_data = application_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
                               'EXT_SOURCE_3', 'DAYS_BIRTH']].copy()

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().sample(10000)#.loc[:100000, :]

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

grid.map_upper(plt.scatter, alpha = 0.2)
grid.map_diag(sns.kdeplot)
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);
plt.suptitle('EXT_SOURCE and DAYS_BIRTH Pair Plot', size = 32, y = 1.05);

Looking at the relationships between DAYS_BIRTH and the three EXT_SOURCEs, we can see the concentration of orange points in each plot where applicants did repay their loan and the blue points where applicants did not repay their loan. There is a small correlation between DAYS_BIRTH and EXT_SOURCE_1 as previously mentioned. EXT_SOURCE_1 here appears to be a good predictor for ability to repay with a clear difference in KDE plots between those who could repay (orange hue) having a high density around 0.25 and those that couldn't repay (blue) having a high density around 0.5 but it should be noted that this feature possesses 173378 missing values. To put this in perspective:

In [ ]:
EXT_SOURCE_1_miss = 173378/len(application_train['EXT_SOURCE_1'])

In [ ]:
print('Percentage of missing values in EXT_SOURCE_1: {:.2f}%'.format(EXT_SOURCE_1_miss))

So this is not a reliable feature since it is often missing. EXT_SOURCE_3 shows a similar distribution so if we look at the missing values there:

In [ ]:
EXT_SOURCE_3_miss = 60965/len(application_train['EXT_SOURCE_3'])

In [ ]:
print('Percentage of missing values in EXT_SOURCE_3: {:.2f}%'.format(EXT_SOURCE_3_miss))

Which is a lot better but there are still a decent amount of missing values in this feature too

EXT_SOURCE_2 has the lowest number of missing values by far (660) but the distribution of values between those who could and could not repay their loans is largely similar. However, the use of all three of these scores will likely be important in prediction despite their seeming to be weak factors since there is some correlation between them and the target

Also an interesting observation between CNT_CHILDREN and DAYS_BIRTH shows that those without children, on average, are older than those with children. These two features showed a negative correlation of -0.33.

In [ ]:
years_birth = -1*application_train['DAYS_BIRTH']/365
years_birth.astype(int)
fig, ax = plt.subplots(figsize = (15,12))
sns.boxplot(application_train['CNT_CHILDREN'], years_birth, ax = ax)
ax.set(xlabel='CNT_CHILDREN', ylabel='Age in years')

Those with children tend to be, on average, younger than those without children. Additionally, those without children and unable to pay back their loans make up the majority of the population:

In [ ]:
g = sns.factorplot(x="CNT_CHILDREN", hue="TARGET", data=application_train,
                   size=6, kind="count", palette="muted")

However, we already know that there is a class imbalance in the data between those who could/could not repay their loans

DAYS_EMPLOYED has an extremely high max so there's likely an error in recording values here:

In [ ]:
application_train['DAYS_EMPLOYED'].describe()

In [ ]:
plt.hist(application_train['DAYS_EMPLOYED'])
plt.ylabel('Frequency')
plt.xlabel('DAYS_EMPLOYED')
plt.title('Distribution of DAYS_EMPLOYED')

365243 likely recorded instead of a missing value so lets replace this with NaN values instead but retain the values in a separate column to ensure that we can later easily find where the values were changed to NaN

Let's make these changes to the entire dataset

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
def application_train_and_test():
    
    application_train = pd.read_csv('data/application_train.csv')
    application_test = pd.read_csv('data/application_test.csv')
    
    application_train = application_train.sort_values(by = 'SK_ID_CURR')
    application_test = application_test.sort_values(by = 'SK_ID_CURR')
    df = application_train.append(application_test).reset_index()

    df['DAYS_EMPLOYED_MISS'] = df['DAYS_EMPLOYED'] == 365243
    df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

    binary_features = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
                       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REG_REGION_NOT_LIVE_REGION',
                       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 
                       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 
                       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7',
                       'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 
                       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
                       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
                       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'DAYS_EMPLOYED_MISS']
    for feature in binary_features:
        df[feature], uniques = pd.factorize(df[feature])

    df, df_cat_col = one_hot_encoder(df)
    df = df.drop('index', axis=1)
    
    del application_train
    del application_test
    
    return df

In [ ]:
df.head() 

# 2. bureau and bureau_balance

In [ ]:
bureau.columns

In [ ]:
bureau.head(10)

SK_ID_BUREAU is the ID of the loan application submitted by a person (represented by SK_ID_CURR). There are multiple loan applications under many SK_ID_CURR candidates. Let's count the amount of loan applications by each person and turn this into a new feature:

In [ ]:
previous_loans = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loans'})
previous_loans.head()

In [ ]:
df = df.merge(previous_loans, on = 'SK_ID_CURR', how = 'left')
df['previous_loans'] = df['previous_loans'].fillna(0)

Similarly, CREDIT_ACTIVE denotes whether the loan is closed or still active. Let's count the number of closed and active loans associated with each individual and see if there is any correlation with the target in our training data:

In [ ]:
closed_loans = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
active_loans = bureau[bureau['CREDIT_ACTIVE'] == 'Active']

closed_loans = closed_loans.groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count().rename(columns = {'CREDIT_ACTIVE': 'closed_loans'})
active_loans = active_loans.groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count().rename(columns = {'CREDIT_ACTIVE': 'active_loans'})

In [ ]:
closed_loans.head()

In [ ]:
active_loans.head()

In [ ]:
df = df.merge(closed_loans, on = 'SK_ID_CURR', how = 'left')
df = df.merge(active_loans, on = 'SK_ID_CURR', how = 'left')

df['closed_loans'] = df['closed_loans'].fillna(0)
df['active_loans'] = df['active_loans'].fillna(0)

In [ ]:
del bureau['CREDIT_ACTIVE']

In [ ]:
df.head()

In [ ]:
bureau.AMT_CREDIT_SUM_OVERDUE.describe()

In [ ]:
bureau['CREDIT_CURRENCY'].value_counts()

The currency for the loans is heavily biased towards currency 1. We will one-hot encode this feature along with others and add it to our training and testing data

In [ ]:
bureau.DAYS_CREDIT.describe()

This looks reasonable

In [ ]:
bureau, bureau_cat_cols = one_hot_encoder(bureau)

bureau_balance, bureau_balance_cat_cols = one_hot_encoder(bureau_balance)

In [ ]:
bureau.head()

Take the min, max, and mean of all the numerical features related to each loan by each SK_ID_CURR and make them new features to merge into train and test sets

In [ ]:
bureau_agg = bureau.groupby('SK_ID_CURR').agg({'DAYS_CREDIT':['min', 'max', 'mean'],
                                               'CREDIT_DAY_OVERDUE':['max', 'mean'],
                                               'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
                                               'DAYS_ENDDATE_FACT': ['mean'],
                                               'AMT_CREDIT_MAX_OVERDUE': ['mean'],
                                               'CNT_CREDIT_PROLONG': ['count'],
                                               'AMT_CREDIT_SUM': ['min', 'max', 'mean'],
                                               'AMT_CREDIT_SUM_DEBT': ['min', 'max', 'mean'],
                                               'AMT_CREDIT_SUM_LIMIT': ['sum', 'mean'],
                                               'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                                               'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],
                                               'AMT_ANNUITY': ['max', 'mean']})

bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

In [ ]:
bureau_agg.head()

In [ ]:
df = df.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

Aggregate features of bureau_balance, join them to bureau table and merge features into train and test:

In [ ]:
bureau_balance_and_bureau = bureau.merge(bureau_balance, on = 'SK_ID_BUREAU', how = 'left')

In [ ]:
bureau_balance_and_bureau.head()

In [ ]:
bureau_balance_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}

for col in bureau_balance_cat_cols:
    bureau_balance_aggregations[col] = ['mean']

bureau_balance_agg = bureau_balance_and_bureau.groupby('SK_ID_CURR').agg(bureau_balance_aggregations)

bureau_balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bureau_balance_agg.columns.tolist()])

In [ ]:
bureau_balance_agg.head()

Set size to NaN e.g. in the scenario where a lone applicant has 4 separate loans with no MONTHS_BALANCE information (paid back in that month presumably), the size of MONTHS_BALANCE will be set to NaN instead of 4

In [ ]:
for row in range(len(bureau_balance_agg)):
    if np.isnan(bureau_balance_agg.iloc[row, 0]) == True and np.isnan(bureau_balance_agg.iloc[row, 1]) == True:
        bureau_balance_agg.iloc[row, 2] = np.nan

Merge these aggregated features into train and test

In [ ]:
df = df.merge(bureau_balance_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

In [ ]:
missing_value_counter(df)

In [ ]:
print(np.shape(df))

# 3. credit_card_balance

In [ ]:
credit_card_balance.head()

In [ ]:
missing_value_counter(credit_card_balance)

In [ ]:
print(np.shape(credit_card_balance))

In [ ]:
credit_card_balance[credit_card_balance['SK_ID_CURR'] == 378907].sort_values(by = 'MONTHS_BALANCE', ascending = False).head()

SK_ID_CURR 378907 had one previous credit card loan with Home-Credit and the above are monthly snapshots of this loan 

Let's create a feature that counts the no. of previous loans by each applicant. The number of months previous that the applicant took out the previous loan (SK_ID_PREV) could also be useful.

In [ ]:
prev_credit_months = credit_card_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index=False)['MONTHS_BALANCE'].count().rename(columns = {'MONTHS_BALANCE': 'prev_credit_months'})

Setting the prev_credit_months column to negative relative to current loan:

In [ ]:
prev_credit_months['prev_credit_months'] = -prev_credit_months['prev_credit_months']

From this DataFrame, we can calculate the number of previous loans associated with each current applicant

In [ ]:
prev_credit_months['SK_ID_CURR'].value_counts()

In [ ]:
previous_credit_loans = pd.DataFrame()
previous_credit_loans['SK_ID_CURR'] = prev_credit_months['SK_ID_CURR'].value_counts().index.values
previous_credit_loans['previous_credit_loans'] = prev_credit_months['SK_ID_CURR'].value_counts().values
previous_credit_loans.head()

In [ ]:
df = df.merge(previous_credit_loans, on = 'SK_ID_CURR', how = 'left')
df['previous_credit_loans'] = df['previous_credit_loans'].fillna(0)

In [ ]:
df.head()

## Number of loans of different status per SK_ID_CURR

In [ ]:
current_loan_status = credit_card_balance[['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS']].sort_values(by = ['SK_ID_CURR', 'NAME_CONTRACT_STATUS']).drop_duplicates()

In [ ]:
current_loan_status['SK_ID_CURR'].value_counts()

In [ ]:
current_loan_status[current_loan_status['SK_ID_CURR']==419475]

In [ ]:
current_loan_status.head()

In [ ]:
current_loan_status[current_loan_status['NAME_CONTRACT_STATUS']=='Signed']

In [ ]:
pd.get_dummies(current_loan_status[current_loan_status['NAME_CONTRACT_STATUS']=='Signed'])

In [ ]:
current_loan_status[current_loan_status['SK_ID_CURR']==125372]

In [ ]:
current_loan_status.NAME_CONTRACT_STATUS.value_counts()

Looking only at Active and Completed loans:

In [ ]:
current_loan_status = current_loan_status[current_loan_status['NAME_CONTRACT_STATUS'].isin(['Active', 'Completed'])]

In [ ]:
current_loan_status['SK_ID_CURR'].value_counts()

In [ ]:
current_loan_status[current_loan_status['SK_ID_CURR']==355767]

In [ ]:
current_loan_status.NAME_CONTRACT_STATUS.value_counts()

### Completed loans

In [ ]:
current_loan_status.sort_values('SK_ID_CURR').head(10)

In [ ]:
prev_credit_completed = current_loan_status[current_loan_status['NAME_CONTRACT_STATUS'] == 'Completed']

In [ ]:
prev_credit_completed.head(20)

In [ ]:
prev_credit_completed = pd.get_dummies(prev_credit_completed)

In [ ]:
del prev_credit_completed['SK_ID_PREV']

In [ ]:
prev_credit_completed.sort_values('SK_ID_CURR').head(20)

In [ ]:
prev_credit_completed['SK_ID_CURR'].value_counts()

In [ ]:
prev_credit_completed = prev_credit_completed.groupby('SK_ID_CURR', as_index=False)['NAME_CONTRACT_STATUS_Completed'].count()

In [ ]:
prev_credit_completed[prev_credit_completed['SK_ID_CURR']==263421]

In [ ]:
df = df.merge(prev_credit_completed, on = 'SK_ID_CURR', how = 'left')
df['NAME_CONTRACT_STATUS_Completed'] = df['NAME_CONTRACT_STATUS_Completed'].fillna(0)

In [ ]:
df.head()

### Active loans

In [ ]:
prev_credit_active = current_loan_status.drop_duplicates(subset = ['SK_ID_CURR', 'SK_ID_PREV'], keep = False)
prev_credit_active = prev_credit_active[prev_credit_active['NAME_CONTRACT_STATUS'] == 'Active']
prev_credit_active = pd.get_dummies(prev_credit_active)

del prev_credit_active['SK_ID_PREV']

prev_credit_active.head()

In [ ]:
prev_credit_active = prev_credit_active.groupby('SK_ID_CURR', as_index=False)['NAME_CONTRACT_STATUS_Active'].count()

df = df.merge(prev_credit_active, on = 'SK_ID_CURR', how = 'left')
df['NAME_CONTRACT_STATUS_Active'] = df['NAME_CONTRACT_STATUS_Active'].fillna(0)

In [ ]:
df.head()

### Creating aggregate features from numerical columns in credit_card_balance and merging into train and test

In [ ]:
credit_card_balance.head()

In [ ]:
credit_card_balance.dtypes

In [ ]:
credit_card_balance.drop(columns = ['NAME_CONTRACT_STATUS', 'SK_ID_PREV'], inplace = True)
cc_agg = credit_card_balance.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
df = df.merge(cc_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

## 4. installments_payments

In [ ]:
installments_payments.head()

In [ ]:
np.shape(installments_payments)

In [ ]:
missing_value_counter(installments_payments)

In [ ]:
installments_payments.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV']).head()

### Mean no. of days it took each applicant to pay installments on previous loans

In [ ]:
installments_day_diff = installments_payments.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV'])[['SK_ID_CURR', 'SK_ID_PREV', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT']]
installments_day_diff['installments_day_diff'] = installments_day_diff['DAYS_INSTALMENT'] - installments_day_diff['DAYS_ENTRY_PAYMENT']
installments_day_diff_agg = installments_day_diff.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index=False)['installments_day_diff'].mean()

In [ ]:
del installments_day_diff_agg['SK_ID_PREV']

In [ ]:
installments_day_diff_agg = installments_day_diff_agg.groupby('SK_ID_CURR', as_index=False)['installments_day_diff'].mean()

In [ ]:
installments_day_diff_agg.head()

In [ ]:
df = df.merge(installments_day_diff_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

In [ ]:
del installments_payments['DAYS_INSTALMENT']
del installments_payments['DAYS_ENTRY_PAYMENT']

In [ ]:
installments_payments.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV']).head()

### Mean difference between installment amount and what was actually paid each month

In [ ]:
installments_diff = installments_payments.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV'])[['SK_ID_CURR', 'SK_ID_PREV', 'AMT_INSTALMENT', 'AMT_PAYMENT']]
installments_diff['installments_diff'] = installments_diff['AMT_INSTALMENT'] - installments_diff['AMT_PAYMENT']
installments_diff_agg = installments_diff.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index=False)['installments_diff'].mean()

In [ ]:
installments_diff_agg.head()

In [ ]:
del installments_diff_agg['SK_ID_PREV']

In [ ]:
installments_diff_agg.head()

In [ ]:
installments_diff_agg = installments_diff_agg.groupby('SK_ID_CURR', as_index=False)['installments_diff'].mean()

In [ ]:
installments_diff_agg.head()

In [ ]:
df = df.merge(installments_diff_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

# 5. POS_CASH_balance

In [ ]:
POS_CASH_balance.head()

In [ ]:
prev_POS_months = POS_CASH_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index=False)['MONTHS_BALANCE'].count().rename(columns = {'MONTHS_BALANCE': 'prev_POS_months'})

In [ ]:
prev_POS_months.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV']).head(20)

Setting the prev_POS_months column to negative relative to current loan:

In [ ]:
prev_POS_months['prev_POS_months'] = -prev_POS_months['prev_POS_months']

In [ ]:
prev_POS_months.head()

From this DataFrame, we can calculate the number of previous loans associated with each current applicant

In [ ]:
prev_POS_months['SK_ID_CURR'].value_counts()

In [ ]:
previous_POS_loans = pd.DataFrame()
previous_POS_loans['SK_ID_CURR'] = prev_POS_months['SK_ID_CURR'].value_counts().index.values
previous_POS_loans['previous_POS_loans'] = prev_POS_months['SK_ID_CURR'].value_counts().values
previous_POS_loans.head()

In [ ]:
df = df.merge(previous_POS_loans, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df['previous_POS_loans'] = df['previous_POS_loans'].fillna(0)

In [ ]:
df.head()

In [ ]:
POS_CASH_balance.sort_values(by = ['SK_ID_CURR', 'SK_ID_PREV']).head(10)

In [ ]:
current_POS_status = POS_CASH_balance[['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS']].sort_values(by = ['SK_ID_CURR', 'NAME_CONTRACT_STATUS']).drop_duplicates()

In [ ]:
current_POS_status.head(10)

In [ ]:
current_POS_status.NAME_CONTRACT_STATUS.value_counts()

Looking only at Active and Completed loans:

In [ ]:
current_POS_status = current_POS_status[current_POS_status['NAME_CONTRACT_STATUS'].isin(['Active', 'Completed'])]

In [ ]:
current_POS_status.head()

### Completed loans

In [ ]:
prev_POS_completed = current_POS_status[current_POS_status['NAME_CONTRACT_STATUS'] == 'Completed']
prev_POS_completed = pd.get_dummies(prev_POS_completed)

del prev_POS_completed['SK_ID_PREV']

prev_POS_completed.head()

In [ ]:
prev_POS_completed = prev_POS_completed.rename(columns = {'NAME_CONTRACT_STATUS_Completed': 'NAME_CONTRACT_STATUS_Completed_POS'})

In [ ]:
prev_POS_completed = prev_POS_completed.groupby('SK_ID_CURR', as_index=False)['NAME_CONTRACT_STATUS_Completed_POS'].count()

In [ ]:
df = df.merge(prev_POS_completed, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

In [ ]:
df['NAME_CONTRACT_STATUS_Completed_POS'] = df['NAME_CONTRACT_STATUS_Completed_POS'].fillna(0)

In [ ]:
df.head()

### Active loans

In [ ]:
prev_POS_active = current_POS_status.drop_duplicates(subset = ['SK_ID_CURR', 'SK_ID_PREV'], keep = False)
prev_POS_active = prev_POS_active[prev_POS_active['NAME_CONTRACT_STATUS'] == 'Active']
prev_POS_active = pd.get_dummies(prev_POS_active)

del prev_POS_active['SK_ID_PREV']

prev_POS_active.head()

In [ ]:
prev_POS_active = prev_POS_active.rename(columns = {'NAME_CONTRACT_STATUS_Active': 'NAME_CONTRACT_STATUS_Active_POS'})

In [ ]:
prev_POS_active = prev_POS_active.groupby('SK_ID_CURR', as_index=False)['NAME_CONTRACT_STATUS_Active_POS'].count()

In [ ]:
df = df.merge(prev_POS_active, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df['NAME_CONTRACT_STATUS_Active_POS'] = df['NAME_CONTRACT_STATUS_Active_POS'].fillna(0)

In [ ]:
df.head()

### Creating aggregate features from numerical columns in POS_CASH_balance and merging into train and test

In [ ]:
POS_CASH_balance.head()

In [ ]:
POS_CASH_balance.dtypes

In [ ]:
POS_CASH_balance.drop(columns = ['NAME_CONTRACT_STATUS', 'SK_ID_PREV'], inplace = True)

In [ ]:
POS_CASH_balance.sort_values(by = ['SK_ID_CURR']).head(20)

In [ ]:
POS_CASH_balance_agg = POS_CASH_balance.groupby('SK_ID_CURR').agg(['min', 'max', 'mean'])

In [ ]:
POS_CASH_balance_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in POS_CASH_balance_agg.columns.tolist()])

In [ ]:
df = df.merge(POS_CASH_balance_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()

# 6. previous_application

In [ ]:
previous_application = pd.read_csv('data/previous_application.csv')

In [ ]:
previous_application.head()

In [ ]:
previous_application.dtypes

In [ ]:
previous_application[['SK_ID_PREV', 'SK_ID_CURR',
                      'DAYS_FIRST_DRAWING','DAYS_FIRST_DUE',
                      'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE',
                      'DAYS_TERMINATION']].sort_values(['SK_ID_PREV', 'SK_ID_CURR']).head(20)

Same issue with 365243.0 as seen before. Let's set all of these to NaN

In [ ]:
previous_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

Let's one-hot encode everything and make aggregate features from the categorical and numerical columns that result

In [ ]:
previous_application[['SK_ID_PREV', 'SK_ID_CURR',
                      'NFLAG_LAST_APPL_IN_DAY','SELLERPLACE_AREA',
                      'HOUR_APPR_PROCESS_START', 'DAYS_DECISION']].sort_values(['SK_ID_PREV', 'SK_ID_CURR']).head(10)

In [ ]:
num_aggregations = {'AMT_ANNUITY': ['min', 'max', 'mean'],
                    'AMT_APPLICATION': ['min', 'max', 'mean'],
                   'AMT_CREDIT': ['min', 'max', 'mean'],
                   'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
                   'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
                   'HOUR_APPR_PROCESS_START': ['mean'],
                   'NFLAG_LAST_APPL_IN_DAY': ['mean', 'size'],
                   'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
                   'RATE_INTEREST_PRIMARY': ['min', 'max', 'mean'],
                   'RATE_INTEREST_PRIVILEGED': ['min', 'max', 'mean'],
                   'DAYS_DECISION': ['min', 'max', 'mean'],
                   'CNT_PAYMENT': ['sum', 'mean']}

In [ ]:
previous_application, cat_cols = one_hot_encoder(previous_application)

In [ ]:
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']

In [ ]:
previous_application.head()

In [ ]:
prev_agg = previous_application.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

In [ ]:
prev_agg.head(10)

In [ ]:
approved = previous_application[previous_application['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APR_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])

In [ ]:
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

In [ ]:
refused = previous_application[previous_application['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REF_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])

In [ ]:
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

In [ ]:
prev_agg.head()

In [ ]:
df = df.merge(prev_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.head()